In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
from itertools import product
import time as t

In [3]:
from da_utils import plot_diff_all, plot_gem0_scan, write_gem0_offline

In [4]:
xlabels = ['te_value', 'ti_value', 'te_ddrho', 'ti_ddrho']
ylabels = ['te_transp_flux', 'ti_transp_flux']

n_fts=8

In [6]:
### To create GEM0 scan files:

date_now = '20240115'

#file_in = "../../muscle3/ref_train_data_5000.csv"
#file_in = "gem0_new_data_20231215.csv"
#file_in = "gem0py_lhc_20240110_all.csv"

# coming back to 5000 full grid GEM0 data
file_in = "gem0_new_data_20231208.csv"

### Read inputs
data = pd.read_csv(file_in)
data_x = data[xlabels]
data_y = data[ylabels]
x = np.array(data_x)

# Create scan files
print("Creating scan files")
modtype="gpr"
remainder_file_path = f"scan_gem0{modtype}_remainder_"
remainder_file_date = date_now
st_t = t.time()

scan_data = [[plot_gem0_scan(x,input_number=i,output_number=o, flag_plot=False, file_name_suf=date_now, extend_factor=0.05, \
                               cut_option='file', remainder_values=f"{remainder_file_path}{xlabels[i]}_{remainder_file_date}", \
                             ) for i in range(len(xlabels))] for o in range(len(ylabels))]
print("Time to create scan files: ", t.time()-st_t)

scan_data_perft = []

# Reshape dictionaries
for n_ft in range(n_fts):
     scan_data_perft.append({(f"{xlabels[i]}_{ylabels[j]}",k): scan_data[j][i][(f"ft{n_ft}",k)] for i,j,k in product(range(len(xlabels)),range(len(ylabels)),['x','y'])})	

# Write down scan files
date_last = '20240115'

for n_ft in range(n_fts):
     s = scan_data_perft[n_ft]
     spd = pd.DataFrame.from_dict(s)
     spd.to_csv(f"scan_gem0py_{date_last}_ft{n_ft}.csv")


Creating scan files
for te_value @ft#0 remainder values are: [ 1781.15627304 -2654.58327936 -3295.23287111]
time to evaluate GEM0 for i0o0f0: 3.4885647296905518
for te_value @ft#1 remainder values are: [ 1302.31441865 -5539.09665125 -3792.55775593]
time to evaluate GEM0 for i0o0f1: 4.410279035568237
for te_value @ft#2 remainder values are: [ 1143.89492147 -6378.02803047 -1219.412089  ]
time to evaluate GEM0 for i0o0f2: 4.781259775161743
for te_value @ft#3 remainder values are: [ 1009.34340847 -5998.75995782 -2246.38414478]
time to evaluate GEM0 for i0o0f3: 5.32145357131958
for te_value @ft#4 remainder values are: [  837.27936622 -3191.1136587  -2204.78987435]
time to evaluate GEM0 for i0o0f4: 5.730129957199097
for te_value @ft#5 remainder values are: [  646.14992543 -3644.59069779 -3022.08719382]
time to evaluate GEM0 for i0o0f5: 6.187575340270996
for te_value @ft#6 remainder values are: [  468.86205074 -3601.71451251 -2689.19692887]
time to evaluate GEM0 for i0o0f6: 6.425935745239258


In [13]:
# # Test: reading a multi-indexed dataframe
# test_df = pd.read_csv("scan_gem0_remainder_te_value.csv", header=[0, 1], index_col=0,)

# test_df = test_df[(f"ft{0}", xlabels[0])]

# test_array = np.array(test_df)
# print(test_array)

[ 1807.38363671 -2654.58327936 -3295.23287111]


In [23]:
# To create surrogate scan files:

#!cd ../../../EasySurrogate && pip install . && cd tests/gem_gp && for((i=0;i<8;i++)); do python3 test_model_ann.py ${i} 20231218 && cp scan_${i}.csv ../../../MFW/uq/basicda/scan_gem0surr_20240105_ft${i}.csv ; done && cd ../../../MFW/uq/basicda

!cd ../../../EasySurrogate && pip install . && cd tests/gem_gp && for((i=0;i<8;i++)); do python3 test_model.py ${i} 20231212 && cp scan_${i}.csv ../../../MFW/uq/basicda/scan_gem0surr_20240105_ft${i}.csv ; done && cd ../../../MFW/uq/basicda


DEPRECATION: Loading egg at /cobra/u/yyudin/.local/lib/python3.11/site-packages/mogp_emulator-0.7.2-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at /cobra/u/yyudin/conda-envs/python3114/lib/python3.11/site-packages/easysurrogate-0.18-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
Processing /cobra/u/yyudin/code/EasySurrogate
  Preparing metadata (setup.py) ... done
  Created wheel for easysurrogate: filename=easysurrogate-0.24.1-py3-none-any.whl size=105687 sha256=2d5b8edc078b729106e29ffce107594d81ab9e81a04f19e7143ab1814b9ccadc
  Stored in directory: /tmp/pip-ephem-wheel-cache-y2hw69w9/wheels/a7/9b/db/505d51a3a6df8379c6b57380cb045f3d3a3237c864e8adaa7c
Successfully built easysurrogate
  Attempting uninstall: easysurrogate
    Found existing installation: easysurrogate 0.24.1
    Uninstal

In [27]:
# To plot the difference between GEM0 and surrogate, and their comparison

date = '20240115'
surtype = 'gpr'

file_pref_gem0 = f"gem0py_{date}"
file_pref_surr = f"gem0{surtype}_{date}"

save_file = f"gem0vs{surtype}_{date}.pdf"

plot_diff_all(file_pref_1=file_pref_gem0, file_pref_2=file_pref_surr, save_file=save_file)

!mv *gem0vsgpr_20240115.pdf ../../..

In [20]:
# To generate a dataset of GEM0 evaluations
#nft = 8
date = '20240110'

save_filename=f"gem0py_lhc_{date}_ft"

for i in range(n_fts):
    
    print(f"flux tube number: {i}")

    write_gem0_offline(n_ft=i, filename=save_filename)


flux tube number: 0
time to create extcodehelper: 0.5018746852874756
time to create LHC-S: 0.0011005401611328125
negative Ti:
[]
time to evaluate GEM0: 3.246825695037842
time to save GEM0: 0.02131485939025879
flux tube number: 1
time to create extcodehelper: 0.46692419052124023
time to create LHC-S: 0.0014145374298095703
negative Ti:
[]
time to evaluate GEM0: 3.8447012901306152
time to save GEM0: 0.020212888717651367
flux tube number: 2
time to create extcodehelper: 0.4581937789916992
time to create LHC-S: 0.001117706298828125
negative Ti:
[]
time to evaluate GEM0: 4.683401107788086
time to save GEM0: 0.018392324447631836
flux tube number: 3
time to create extcodehelper: 0.46006011962890625
time to create LHC-S: 0.0009589195251464844
negative Ti:
[]
time to evaluate GEM0: 4.644847631454468
time to save GEM0: 0.01986551284790039
flux tube number: 4
time to create extcodehelper: 0.44414734840393066
time to create LHC-S: 0.0010786056518554688
negative Ti:
[]
time to evaluate GEM0: 5.09802

In [21]:
# temporary: merge generated csv file into one, adding a column with the flux tube number

import glob

date = '20240110'
path = f"gem0py_lhc_{date}_ft*.csv" 

all_files = glob.glob(path)
df_from_each_file = [pd.read_csv(f) for f in all_files]

for i in range(len(df_from_each_file)):
    df_from_each_file[i]['ft'] = i

df_merged = pd.concat(df_from_each_file, ignore_index=True, sort=False)
df_merged.to_csv( f"gem0py_lhc_{date}_all.csv")
